# TFIDFによる文書類似度推定

In [2]:
import urllib
import glob
import re
import os
from typing import List, Tuple

import MeCab
import numpy as np
from IPython.display import Image
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
# configs
CORPUS_PATH = 'corpus.txt'
m = MeCab.Tagger("-Owakati")  # TODO:わかち書きしているので、品詞指定で特徴語を抽出するようにする必要がありそう
DATASET_PATH = '/Users/rikeda/Google ドライブ/Colab Notebooks/nikkei'
ILLUST_YA_CSV_PATH = '/Users/rikeda/Downloads/hackday/illust.csv'

### いらすとやの見出しの格納

In [4]:
# 試しにぐっちが集めたいらすとやの見出しを使ってみる
import csv
with open(ILLUST_YA_CSV_PATH, 'r') as f:
    reader = csv.reader(f)
    illusts = [(m.parse(row[1]), row[2]) for row in reader]
print(illusts[:3])

[('お金 の 単位 の キャラクター （ 強い 円 ） \n', 'https://4.bp.blogspot.com/-jN5C5Y4xN9s/V2zF20D4INI/AAAAAAAA8AQ/sVYu-ezC_3UKtX4n75AaHUqEskfQQnS7QCLcB/s150/money_character_strong_yen.png'), ('風邪 薬 の イラスト \n', 'https://2.bp.blogspot.com/-57P4j4ba84E/VyNdckfKDVI/AAAAAAAA6No/00EjX61RCQwiFfvQjglB7M55X2xsO65MQCLcB/s400/medicine_kaze.png'), ('青い バラ の イラスト \n', 'https://3.bp.blogspot.com/-D1ojODeU7lg/U0pS6kDirII/AAAAAAAAe_Y/LU00iN_kGs0/s400/rose_blue.png')]


In [5]:
def replace_num_to_zero(text: str) -> str:
    return re.sub(r'[0-9]+', '0', text)

def format_text(content: List[str]) -> List[str]:
    """
    TODO:わかち書きしているので、品詞指定で特徴語を抽出するようにする必要がありそう
    """
    
    while '\n' in content:
        content.remove('\n')
    while '' in content:
        content.remove('')
    # 記者名を削除
    if content[-1][0] == '（':
        content = content[:-1]
    return ' '.join([m.parse(replace_num_to_zero(line.replace('\n', ''))).strip() for line in content[1:]])

## データセットの読み込み
今回は、自分がクローリングしていた日経新聞の記事、135,639記事を使用

In [6]:
corpus = []
if os.path.exists(CORPUS_PATH):
    with open(CORPUS_PATH, 'r') as f:
        for line in f:
            corpus.append(line)
else:
    for cpath in glob.glob(f'{DATASET_PATH}/*'):
        for dpath in glob.glob(f'{cpath}/*'):
            for text in glob.glob(f'{dpath}/*'):
                with open(text, 'r') as f:
                    lines = f.readlines()
                    if len(lines) > 0:
                        lines = format_text(lines)
                        corpus.append(lines)
    with open('corpus.txt', 'w') as f:
        f.write("\n".join(corpus))

In [7]:
len(corpus)

135639

## TF-IDFの実装

In [8]:
# slothlib導入
# ぐっちのコードを参照
slothlib_path = 'http://svn.sourceforge.jp/svnroot/slothlib/CSharp/Version1/SlothLib/NLP/Filter/StopWord/word/Japanese.txt'
slothlib_file = urllib.request.urlopen(slothlib_path)
slothlib_stopwords = [line.decode("utf-8").strip() for line in slothlib_file]
slothlib_stopwords = [ss for ss in slothlib_stopwords if not ss==u'']
slothlib_stopwords.extend(["", "0", "は", "を", "と", "に", "へ", "が", "の", 
                           "へ", "と", "より", "から", "にて", "して", "ば", 
                           "とも", "ど", "ども", "が", "に", "を", "て", 
                           "して", "で", "ながら", "ものを", "すら", "し", 
                           "のみ", "ばかり", "など", "まで", "は", "も", 
                           "でも", "しか", "さえ", "こそ", "な", "に", 
                           "とも", "の", "か", "い", "た", "として", 
                           "や", "いる", "だ", "った", "さ", "れ", "とともに", "です", "にわたり", "か月"])

In [37]:
# TF-IDFの演算
# TODO: analyzerをmecabのオブジェクトにすることで、vectorizerに分かち書き前のテキストを入れても成立するようになる
vectorizer = TfidfVectorizer(use_idf=True, max_features=10000, analyzer='word', ngram_range=(1, 1), stop_words=slothlib_stopwords)
vecs = vectorizer.fit_transform(corpus)

In [38]:
sentence = "最年少 で の 七 大 タイトル を 獲得 し た 芝野 虎 丸新 名人 （ 0 日 夜 、 静岡 県 熱海 市 ）"
sentence = "ゾンビ 映画 の 巨匠 と いわ れる ジョージ ・ A ・ ロメ ロ 監督 によって ホラー 映画 として 発展 し た が 、 近年 は ドラマ や ミュージカル など 様々 な ジャンルで 、 多様 な ゾンビ が 描か れ て き た 。"
sentence = m.parse("東京都内で２５日、新型コロナウイルスの感染者が新たに４８人確認されたことがわかった。１日当たりの感染者は２４日（５５人）を下回ったが、直近１週間の平均は約３９人と高水準が続いている。").replace(' \n', '')
a = vectorizer.transform([sentence])
b = vectorizer.transform([illust[0] for illust in illusts])
similarity = cosine_similarity(a, b)[0]
top3_indices = np.argsort(similarity)[::-1][:3]

In [39]:
# top-1
print(similarity[top3_indices.tolist()[0]])
print(illusts[top3_indices.tolist()[0]][0])
Image(url =  illusts[top3_indices.tolist()[0]][1])

0.4273318628435663
母子 感染 ・ 垂直 感染 の イラスト 



In [40]:
# top-2
print(similarity[top3_indices.tolist()[1]])
print(illusts[top3_indices.tolist()[1]][0])
Image(url =  illusts[top3_indices.tolist()[1]][1])

0.389017408642666
地域 を またい だ 感染 拡大 の イラスト （ 感染 地域 から ） 



In [41]:
# top-3
print(similarity[top3_indices.tolist()[2]])
print(illusts[top3_indices.tolist()[2]][0])
Image(url =  illusts[top3_indices.tolist()[2]][1])

0.36421959965615347
蚊 が 媒介 する 感染 症 の イラスト 



## 学習済みTF-IDFデータの保存

In [17]:
import pickle
with open('tfidf.pkl', 'wb') as f:
    pickle.dump(vectorizer, f)

In [18]:
b.shape
a.shape

(1, 10000)

In [32]:
# 呼び出し方
with open('tfidf.pkl', 'rb') as f:
    tfidf_vectorizer = pickle.load(f)
a = vectorizer.transform([sentence])
b = vectorizer.transform([illust[0] for illust in illusts])
similarity = cosine_similarity(a, b)[0]
top3_indices = np.argsort(similarity)[::-1][:3]

print(similarity[top3_indices.tolist()[0]])
print(illusts[top3_indices.tolist()[0]][0])
Image(url =  illusts[top3_indices.tolist()[0]][1])

0.4273318628435663
母子 感染 ・ 垂直 感染 の イラスト 



意味不明な結果にwww

## 次元削減

In [33]:
from sklearn.decomposition import TruncatedSVD

dim = 200
lsa = TruncatedSVD(n_components=dim)
lsa.fit_transform(vecs)

array([[ 0.20057168,  0.04101422, -0.02305884, ..., -0.01248164,
         0.01372317, -0.00641515],
       [ 0.21492051,  0.20413429,  0.06769231, ...,  0.03865262,
         0.01295171,  0.00026097],
       [ 0.16801205,  0.15235989,  0.02243906, ..., -0.02759267,
        -0.00453862,  0.01738552],
       ...,
       [ 0.20871139,  0.03475033, -0.05776242, ..., -0.00053156,
         0.00123192,  0.01133208],
       [ 0.0235006 , -0.0020007 , -0.01542956, ...,  0.01720896,
        -0.00092372, -0.0133614 ],
       [ 0.07169848, -0.00827597, -0.04076585, ...,  0.0033136 ,
        -0.01909184, -0.00784076]])

In [34]:
# 性能変化の確認
a = lsa.transform(a)
b = lsa.transform(b)
similarity = cosine_similarity(a, b)[0]
top3_indices = np.argsort(similarity)[::-1][:3]

print(similarity[top3_indices.tolist()[0]])
print(illusts[top3_indices.tolist()[0]][0])
Image(url =  illusts[top3_indices.tolist()[0]][1])

0.8505344413896647
母子 感染 ・ 垂直 感染 の イラスト 



In [35]:
print(similarity[top3_indices.tolist()[1]])
print(illusts[top3_indices.tolist()[1]][0])
Image(url =  illusts[top3_indices.tolist()[1]][1])

0.8405352122142054
いろいろ な 感染 症 予防 の イラスト 文字 （ 動物 ・ て を あらお う ） 



In [36]:
print(similarity[top3_indices.tolist()[2]])
print(illusts[top3_indices.tolist()[2]][0])
Image(url =  illusts[top3_indices.tolist()[2]][1])

0.8239409631680794
罠 に かかっ た イノシシ の イラスト 



In [22]:
with open('lsa_for_tfidf.pkl', 'wb') as f:
    pickle.dump(lsa, f)